# Mandatory Exercise - Session 5

### Students: Nafis Banirazi & Jan Carbonell

### Lab Objective:
The Objective of this lab is to categorize the given pairs, print their most frequent WordNet synset, their corresponding least common subsumer (LCS) and their similarity using the following functions:

- Path Similarity
- Leacock-Chodorow Similarity
- Wu-Palmer Similarity
- Lin Similarity


In [1]:
# initial imports. Could also be done in the PC
import nltk
import numpy as np
import pandas as pd
import itertools

#additional set of imports
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic


#given set of pairs
pairs = [('the', 'DT'), ('man', 'NN'), ('swim', 'VB'), \
         ('with', 'PR'), ('a', 'DT'), ('girl', 'NN'), \
         ('and', 'CC'), ('a', 'DT'), ('boy', 'NN'), \
         ('whilst', 'PR'), ('the', 'DT'), ('woman', 'NN'), \
         ('walk', 'VB')]
n = {}
v = {}
aux = {}
freq = []
lcs = []
definition = []

Now, for each pair, we will search for their most frequent WordNet sysnset. In the documentation we can find that there are also adjectives and adverbs listed as options but are not used in this given set of pairs. Listing it as a reference: https://wordnet.princeton.edu/documentation/wn1wn

In [2]:
for e in pairs:
    if e[0] not in n and e[0] not in v:
        if e[1] == 'NN':
            n[e[0]] = wn.synset(e[0]+'.n.01')
        elif e[1] == 'VB':
            v[e[0]] = wn.synset(e[0]+'.v.01')
            
#verification that it is properly stored
for keys,values in n.items():
    print(keys, values)
    
for keys,values in v.items():
    print(keys, values)
            

man Synset('man.n.01')
girl Synset('girl.n.01')
boy Synset('male_child.n.01')
woman Synset('woman.n.01')
swim Synset('swim.v.01')
walk Synset('walk.v.01')


For each of the pairs that was found to be on WordNet, we now get their corresponding least commmon subsumer. That is the most specific common ancestor (hypernym) of two concepts found in a given ontology. For example, the LCS of moose and kangaroo in WordNet is mammal.

In [3]:
for key in n:
    freq.append([key,n[key], 'noun'])

for key in v:
    freq.append([key,v[key], 'verb'])

In [4]:
for key in freq:
    row = []
    for alt in freq:
        if key[2] == alt [2]:
            row.append(key[1].lowest_common_hypernyms(alt[1]))
        else:
            row.append(0)
    lcs.append(row)

In [5]:
lcs_np = np.array(lcs)
label = [i[0] for i in freq]
pd.DataFrame(lcs_np, columns = label, index= label)

,man,girl,boy,woman,swim,walk
man,[Synset('man.n.01')],[Synset('adult.n.01')],[Synset('male.n.02')],[Synset('adult.n.01')],0,0
girl,[Synset('adult.n.01')],[Synset('girl.n.01')],[Synset('person.n.01')],[Synset('woman.n.01')],0,0
boy,[Synset('male.n.02')],[Synset('person.n.01')],[Synset('male_child.n.01')],[Synset('person.n.01')],0,0
woman,[Synset('adult.n.01')],[Synset('woman.n.01')],[Synset('person.n.01')],[Synset('woman.n.01')],0,0
swim,0,0,0,0,[Synset('swim.v.01')],[Synset('travel.v.01')]
walk,0,0,0,0,[Synset('travel.v.01')],[Synset('walk.v.01')]


#calculating min value of the algorithms with words only connected by entity
min1 = wn.lch_similarity(a, b)
min2 = wn.lch_similarity(a, b)
min3 = wn.wup_similarity(a, b)
min4 = a.lin_similarity(b, semcor_ic)

# initializing two words only connected by entity
a = wn.synset('plantation.n.01')
b = wn.synset('dinosaur.n.01')
relation = a.lowest_common_hypernyms(b)

#max and min done outside the loop to preserve as we loop through elements

dog = wn.synset('plantation.n.01')
swim = wn.synset('dinosaur.n.01')
a = dog.lowest_common_hypernyms(swim)
b = wn.lch_similarity(dog, swim)
print(a, b)

In [6]:
semcor_ic = wordnet_ic.ic('ic-semcor.dat')

path, lch, wup, lin = [], [], [], []
v1, v2, v3, v4 = 0, 0, 0, 0
max1, max2, max3, max4 = 0, 0, 0, 0
min1, min2, min3, min4 = 0, 0, 0, 0


for key in freq:
    
    #initializing the rows of the matrices
    row1 = []
    row2 = []
    row3 = []
    row4 = []
    
    # calculating max values as the result of running the algorithms on the same element
    max1 = wn.path_similarity(key[1], key[1])
    max2 = wn.lch_similarity(key[1], key[1])
    max3 = wn.wup_similarity(key[1], key[1])
    max4 = key[1].lin_similarity(key[1], semcor_ic)
    
    for alt in freq:
        # adding only if they belong to the same class 'noun' // 'verb'
        if key[2] == alt[2]:
            
            # calculating the result of the algorithm
            v1 = wn.path_similarity(key[1], alt[1])
            v2 = wn.lch_similarity(key[1], alt[1])
            v3 = wn.wup_similarity(key[1], alt[1])
            v4 = key[1].lin_similarity(alt[1], semcor_ic)
            
            # tracking the minimum value in the matrix
            if min1 > v1:
                min1 = v1
            elif min2 > v2:
                min2 = v2
            elif min3 > v3:
                min3 = v3
            elif min4 > v4:
                min4 = v4
            
            # rounding it and appending it to row
            row1.append(round(v1, 3))
            row2.append(round(v2, 3))
            row3.append(round(v3, 3))
            row4.append(round(v4, 3))
            
        else:
            row1.append(0)
            row2.append(0)
            row3.append(0)
            row4.append(0)
    path.append(row1)
    lch.append(row2)
    wup.append(row3)
    lin.append(row4)
    
print(path)
print('\n')
print(lch)
print('\n')
print(wup)
print('\n')
print(lin)
print('\n')


KeyboardInterrupt: 

now we normalize it

In [ ]:
for row in freq:
    
    #initializing the rows of the matrices
    row1 = []
    row2 = []
    row3 = []
    row4 = []
    
    for alt in freq:           
        if key[2] == alt[2]:
            # normalizing, rounding it to three decimals and appending it to row
            (round((v1-min1)/max1, 3))
            row2.append(round((v2-min2)/max2, 3))
            row3.append(round((v3-min3)/max3, 3))
            row4.append(round((v4-min4)/max4, 3))
    path.append(row1)
    lch.append(row2)
    wup.append(row3)
    lin.append(row4)
    
print(path)
print('\n')
print(lch)
print('\n')
print(wup)
print('\n')
print(lin)
print('\n')

<b>Which model would you select?</b> Justify the answer.

Based on the initial graph, both CRF and PER seem like the better performing algorithms. If we could only make a decision based on this data, we would pick the Perceptron model.

In order to pick the best overall model, another relevant measure is the speed of the system. Because of this, we also have implemented a timer in between each of the models and will plot it accordingly to figure out which algorithms were more effective. 

In [ ]:
path_np = np.array(path)
pd.DataFrame(path_np, columns = label, index= label)

In [ ]:
lch_np = np.array(lch)
pd.DataFrame(lch_np, columns = label, index= label)

In [ ]:
wup_np = np.array(wup)
pd.DataFrame(wup_np, columns = label, index= label)

In [ ]:
lin_np = np.array(lin)
pd.DataFrame(lin_np, columns = label, index= label)

In this case, PER seems to be the most effective -accuracy wise- and second most time efficient algorithm. After having performed this additional execution time analysis <b> we are reinsured in our conclusion to pick the Perceptron Model.</b> 

## Conclusions
Over the development of this lab, we have implemented 4 different POS Models, tested them with different segments of the treebank corpus and trained them with another set of segments. From those results, we have plotted their performanced based on accuracy and execution time and have selected the best performing model for this specific case, which happened to be the Perceptron Model. 

Another thing to highlight is that the HiddenMarkovModel looks really efficient timewise. It is likely that its just not in the desired set of conditions regarding the amount of data to perform correctly. 
